In [1]:
pip install fastembed


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from fastembed import TextEmbedding

In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [4]:
query = "I just discovered the course. Can I join now?"

In [5]:
embedding_model = TextEmbedding(model_name=model_handle)

In [6]:
embedding = list(embedding_model.embed([query]))[0]

In [7]:
import numpy as np

print("Embedding shape:", embedding.shape)
print("Minimal value:", np.min(embedding))

Embedding shape: (512,)
Minimal value: -0.11726373885183883


In [8]:
import numpy as np
np.linalg.norm(embedding)

np.float64(1.0)

In [9]:
embedding.dot(embedding)

np.float64(1.0000000000000002)

In [10]:
doc = 'Can I still join the course after the start date?'

In [11]:
embedding2 = list(embedding_model.embed([doc]))[0]

In [12]:
embedding.dot(embedding2)

np.float64(0.9008528895674548)

In [13]:
documents = [
    {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
     'section': 'General course-related questions',
     'question': 'Course - Can I still join the course after the start date?',
     'course': 'data-engineering-zoomcamp'},
    {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
     'section': 'General course-related questions',
     'question': 'Course - Can I follow the course after it finishes?',
     'course': 'data-engineering-zoomcamp'},
    {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
     'section': 'General course-related questions',
     'question': 'Course - When will the course start?',
     'course': 'data-engineering-zoomcamp'},
    {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
     'section': 'General course-related questions',
     'question': 'Course - What can I do before the course starts?',
     'course': 'data-engineering-zoomcamp'},
    {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
     'section': 'General course-related questions',
     'question': 'How can we contribute to the course?',
     'course': 'data-engineering-zoomcamp'}
]

In [14]:
doc_texts = [d['text'] for d in documents] # extract texts
doc_vecs = np.vstack(list(embedding_model.embed(doc_texts))) # embed all texts

In [15]:
doc_norms = np.linalg.norm(doc_vecs, axis=1, keepdims=True)

In [16]:
q_norm = np.linalg.norm(embedding2)

In [17]:
cosine = (doc_vecs @ embedding2) / (doc_norms[:,0] * q_norm)

In [18]:
best_idx = int(np.argmax(cosine))

In [19]:
best_idx, cosine[best_idx], cosine

(1,
 np.float64(0.8498906589665974),
 array([0.81060571, 0.84989066, 0.79913355, 0.68130469, 0.67392881]))

In [20]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [21]:
doc_vecs_new = np.vstack(list(embedding_model.embed(full_text))) # embed all texts

In [22]:
doc_norms_new = np.linalg.norm(doc_vecs_new, axis=1, keepdims=True)

In [23]:
cosine_new = (doc_vecs_new @ embedding2) / (doc_norms_new[:,0] * q_norm)

In [24]:
best_idx_new = int(np.argmax(cosine_new))

In [25]:
best_idx_new, cosine_new[best_idx_new], cosine_new

(0,
 np.float64(0.9273252247514899),
 array([0.92732522, 0.85948475, 0.83966023, 0.76320295, 0.76044493]))

In [26]:
small_model_handle = "BAAI/bge-small-en"

In [27]:
embedding_model_new = TextEmbedding(model_name=small_model_handle)

In [28]:
embedding_new = list(embedding_model_new.embed([query]))[0]


In [29]:
print("Embedding shape:", embedding_new.shape)

Embedding shape: (384,)


In [34]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [31]:
from qdrant_client import QdrantClient, models

In [32]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [33]:
# Define the collection name
collection_name = "homework_module2"

if collection_name in [c.name for c in client.get_collections().collections]:
    client.delete_collection(collection_name)

EMBEDDING_DIMENSIONALITY = 384  # Dimensionality of the embeddings

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the embeddings
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)   

True

In [35]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question']+ ''+ doc['text'], model=small_model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [36]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [37]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=small_model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [38]:
search_results = search(query, limit=3)

In [39]:
search_results

QueryResponse(points=[ScoredPoint(id=449, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=441, version=0, score=0.8691885, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=448, version=0, score=0.8683311, payload={'text': "Here’s how yo